# Artur Kompała

# Zadanie 6

Wyznacz z pomocą Z3Py co najmniej jedną wartość parametru c, dla której poniższa funkcja:

```c++
unsigned int hash(const char* s, unsigned int c) {
    unsigned int x = 0;
    for (unsigned int i = 0; s[i] != 0; ++i) {
        x <<= 3;
        x ^= s[i];
    }
    return (c/x) % 41;
}
```

będzie doskonałą funkcją haszującą (czyli przyporządkowującą każdemu elementowi liczbę całkowitą bez kolizji) dla zbioru słów kluczowych języka Python:

    and         del           from        not       while
    as          elif          global      or        with
    assert      else          if          pass      yield
    break       except        import      print
    class       exec          in          raise
    continue    finally       is          return
    def         for           lambda      try

## 1. Wstęp
Celem zadania jest znalezienie takiej wartości parametru `c` (typu `unsigned int`), dla której podana funkcja haszująca w C++:
$$h(s) = (c \div x) \pmod{41}$$
będzie **funkcją doskonałą** (różnowartościową, bez kolizji) dla zbioru 31 słów kluczowych języka Python. Zmienna `x` jest wynikiem operacji bitowych na znakach danego słowa.

## 2. Opis proponowanego podejścia
Problem został zamodelowany jako problem spełnialności więzów (SMT) przy użyciu biblioteki Z3Py.

Rozwiązanie podzielono na dwa etapy:
1.  **Pre-kalkulacja (Python):** Wartość `x` w funkcji C++ zależy wyłącznie od liter w słowie, a nie od szukanego parametru `c`. Aby odciążyć solwer, obliczamy wartości `x` dla wszystkich słów kluczowych w czystym Pythonie, symulując zachowanie typów bez znaku (`unsigned int`).
2.  **Rozwiązywanie (Z3):**
    * Definiujemy zmienną symboliczną `c` jako 32-bitowy wektor bitowy (`BitVec`).
    * Dla każdego słowa budujemy wyrażenie haszujące, używając operatorów dzielenia bez znaku (`UDiv`) oraz reszty z dzielenia bez znaku (`URem`), aby wiernie odwzorować zachowanie C++.
    * Nakładamy więz globalny `Distinct`, który wymusza unikalność wszystkich wyników haszowania.
    * Uruchamiamy solwer w celu znalezienia modelu spełniającego założenia.

In [1]:
from z3 import *

# 1. Definicja danych wejściowych (Słowa kluczowe Python)
keywords = [
    "and", "del", "from", "not", "while",
    "as", "elif", "global", "or", "with",
    "assert", "else", "if", "pass", "yield",
    "break", "except", "import", "print",
    "class", "exec", "in", "raise",
    "continue", "finally", "is", "return",
    "def", "for", "lambda", "try"
]

# 2. Funkcja pomocnicza symulująca obliczanie 'x' (zgodnie z kodem C++)
def precompute_x(s):
    """
    Symuluje pętlę C++:
    for (unsigned int i = 0; s[i] != 0; ++i) { x <<= 3; x ^= s[i]; }
    Zwraca wartość x jako liczbę całkowitą.
    """
    x = 0
    for char in s:
        x = (x << 3)
        x = x ^ ord(char)
        # Maskowanie do 32 bitów (symulacja unsigned int)
        x = x & 0xFFFFFFFF
    return x

# Obliczenie wartości x dla każdego słowa (etap pre-processingu)
x_values = {word: precompute_x(word) for word in keywords}

# Sprawdzenie bezpieczeństwa (czy x nie jest zerem)
if any(x == 0 for x in x_values.values()):
    raise ValueError("Błąd: Jedno ze słów generuje x=0, co uniemożliwia dzielenie.")

# 3. Konfiguracja Solvera Z3
solver = Solver()

# Zmienna decyzyjna 'c' - 32-bitowy unsigned int
c = BitVec('c', 32)

# Lista przechowująca wyrażenia symboliczne haszy dla każdego słowa
hash_exprs = []

print("Budowanie modelu w Z3...")

for word in keywords:
    # Pobieramy obliczoną wcześniej wartość x
    x_val = BitVecVal(x_values[word], 32)

    # Modelujemy operację w C++: return (c/x) % 41;
    # Używamy UDiv (Unsigned Division) i URem (Unsigned Remainder)
    # Zwykłe '/' w Z3 na BitVec to dzielenie ze znakiem (signed)!
    h = URem(UDiv(c, x_val), BitVecVal(41, 32))

    hash_exprs.append(h)

# 4. Dodanie więzów (Constraints)
# Więz Distinct wymusza, aby każdy element listy miał unikalną wartość (brak kolizji)
solver.add(Distinct(hash_exprs))

# Opcjonalnie: Szukamy c > 0
solver.add(c > 0)

# 5. Rozwiązywanie i pobieranie wyników
result_status = solver.check()

if result_status == sat:
    model = solver.model()
    c_found = model[c].as_long()
    print(f"\n[SUKCES] Znaleziono rozwiązanie!")
    print(f"Parametr c (int): {c_found}")
    print(f"Parametr c (hex): {hex(c_found)}")
else:
    print("\n[BŁĄD] Nie znaleziono rozwiązania (unsat).")

Budowanie modelu w Z3...

[SUKCES] Znaleziono rozwiązanie!
Parametr c (int): 1948027878
Parametr c (hex): 0x741c8be6


### Weryfikacja rozwiązania:

In [2]:
# 6. Weryfikacja rozwiązania i prezentacja wyników
if result_status == sat:
    print(f"\n{'SŁOWO':<12} | {'x':<10} | {'HASH':<5}")
    print("-" * 35)

    used_hashes = set()
    collisions = 0

    for word in keywords:
        x = x_values[word]
        # Implementacja wzoru haszującego w Pythonie
        h = (c_found // x) % 41

        # Sprawdzanie kolizji
        if h in used_hashes:
            collisions += 1
            marker = "(!)"
        else:
            used_hashes.add(h)
            marker = ""

        print(f"{word:<12} | {x:<10} | {h:<5} {marker}")

    print("-" * 35)
    print(f"Liczba słów: {len(keywords)}")
    print(f"Liczba unikalnych haszy: {len(used_hashes)}")

    if collisions == 0 and len(used_hashes) == len(keywords):
        print("\nWNIOSEK: Funkcja jest doskonała. Wszystkie hasze są unikalne.")
    else:
        print(f"\nWNIOSEK: Wykryto {collisions} kolizji.")


SŁOWO        | x          | HASH 
-----------------------------------
and          | 6996       | 17    
del          | 6724       | 6     
from         | 54165      | 7     
not          | 6284       | 38    
while        | 506181     | 35    
as           | 891        | 13    
elif         | 53806      | 1     
global       | 3507684    | 22    
or           | 778        | 21    
with         | 63368      | 32    
assert       | 3624100    | 4     
else         | 54013      | 26    
if           | 814        | 25    
pass         | 64427      | 19    
yield        | 481348     | 29    
break        | 450083     | 23    
except       | 3529908    | 18    
import       | 3319844    | 12    
print        | 523588     | 30    
class        | 455595     | 11    
exec         | 55115      | 2     
in           | 806        | 40    
raise        | 523197     | 33    
continue     | 233605709  | 8     
finally      | 27965977   | 28    
is           | 827        | 3     
return       | 4171

## 3. Wnioski

1.  Udało się wyznaczyć wartość parametru `c` 1948027878, która gwarantuje brak kolizji dla podanego zbioru słów kluczowych.
2.  Zastosowanie Z3Py pozwoliło na automatyczne odnalezienie stałej bez konieczności stosowania algorytmów brute-force, co przy 32-bitowej przestrzeni poszukiwań ($2^{32}$ możliwości) byłoby czasochłonne.
3.  Kluczowe dla poprawnego rozwiązania było użycie operatorów `UDiv` i `URem`, ponieważ standardowe operatory w Z3 mogą interpretować wektory bitowe jako liczby ze znakiem (signed), co jest niezgodne z typem `unsigned int` w kodzie C++.
4.  Otrzymana funkcja haszująca pozwala na rozpoznawanie słów kluczowych w czasie stałym $O(1)$ przy użyciu tablicy o rozmiarze 41.
5. Otrzymaną wartość parametru c sprawdzono również w oryginalnej implementacji w C++.
Dla wszystkich 31 słów kluczowych nie wystąpiły kolizje, co potwierdza poprawność modelu w Z3.